In [4]:
import numpy as np 
from scipy.integrate import odeint
import os, sys, warnings
from pathlib import Path
from os.path import dirname, realpath
script_dir = Path(dirname(realpath('.')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
import utility as ut
import surrogate_nn as srnn
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
warnings.filterwarnings('ignore')

In [5]:
L63_data_path = '../data/L63-trajectories'
save_folder='../data/rf-start'
log_interval = 100
milestones = [10*2**n for n in range(15)]
learning_rate = 1e-3
drop = 0.7
steps = int(1e4)
save_interval = 100
N = 20000
L0 = 0.4
L1 = 3.5
beta = 4e-5
partition = [100, 100, 100]
train = np.load(f'{L63_data_path}/train.npy').astype(np.float32)
test = np.load(f'{L63_data_path}/test.npy')[:, :, :1000].astype(np.float32)

model = srnn.SurrogateModel_NN(3, 300, name='nn', save_folder=save_folder)
model.init_with_rf(L0, L1, beta, train, partition)
# model.learn(train[:, :N], steps, learning_rate, beta, log_interval, save_interval, milestones, drop, batch_size=N-1)

In [6]:
iters = list(range(0, 10000, 100))
# tau = np.zeros(len(iters))
# for i, iter in enumerate(iters):
#     model.net = torch.load(f'{model.save_folder}/nn_{iter}')
#     tau_f_rmse, tau_f_se, rmse, se = model.compute_tau_f(test[:100], error_threshold=0.05)
#     tau[i] = tau_f_se.mean()
#     print(i)

In [7]:
# plt.plot(iters, tau, label="test_model")
# plt.xlabel('iteration (descent step)')
# plt.ylabel(r'mean $\tau_f$')
# plt.legend()
# plt.savefig(f'{model.save_folder}/iter_vs_mean_tau_f.png')

In [16]:
model.init_with_rf(L0, L1, beta, train, partition=[200, 50, 50])
tau_f_rmse, tau_f_se, rmse, se = model.compute_tau_f(test[:100], error_threshold=0.05)

Time taken by compute_tau_f is 1.3323 seconds


In [17]:
tau_f_se.mean()

4.9635039999999995

In [18]:
model.learn(train[:, :N], 10000, 1e-4, drop=1., batch_size='GD')
tau_f_rmse, tau_f_se, rmse, se = model.compute_tau_f(test[:100], error_threshold=0.05)

step: 0    loss: 4.901759     time elapsed=0.0417
step: 100    loss: 5.666841     time elapsed=1.8259
step: 200    loss: 5.102170     time elapsed=3.6074
step: 300    loss: 5.008341     time elapsed=5.3776
step: 400    loss: 4.962396     time elapsed=7.1766
step: 500    loss: 4.937736     time elapsed=8.9456
step: 600    loss: 4.923184     time elapsed=10.7296
step: 700    loss: 4.913925     time elapsed=12.5208
step: 800    loss: 4.907634     time elapsed=14.3139
step: 900    loss: 4.903021     time elapsed=16.1230
step: 1000    loss: 4.899535     time elapsed=17.8936
step: 1100    loss: 4.896765     time elapsed=19.6585
step: 1200    loss: 4.894525     time elapsed=21.4234
step: 1300    loss: 4.892628     time elapsed=23.2300
step: 1400    loss: 4.891066     time elapsed=25.0172
step: 1500    loss: 4.889759     time elapsed=26.7800
step: 1600    loss: 11.236066     time elapsed=28.5805
step: 1700    loss: 4.890207     time elapsed=30.3955
step: 1800    loss: 4.888700     time elapsed

In [19]:
tau_f_se.mean()

1.9262880000000004

In [9]:
model.net(next(model.dataloader.__iter__())[0])

tensor([[ -9.2252,  -3.4536,  44.8248],
        [-11.8170, -19.2823,  31.4022],
        [  2.4676,   2.5145,  23.6541],
        [  7.8561,  10.0131,  23.5841],
        [  6.3446,  10.8706,  13.8709]], grad_fn=<MmBackward0>)

In [10]:
train[:N]

array([[ -8.361066 ,  -9.38598  , -10.454209 , ...,  -5.5973125,
         -4.3846292,  -3.3684328],
       [-13.313976 , -14.654672 , -15.823077 , ...,   0.951339 ,
          1.1862783,   1.2309328],
       [ 18.146687 ,  19.625534 ,  21.55841  , ...,  31.42136  ,
         29.684444 ,  28.051142 ]], dtype=float32)